#### 本文完全摘录自 https://mp.weixin.qq.com/s/xsEQpjAl_2t-hpcNoMFjgg 
更多内容请点击链接阅读原文

#### 一、分析代码运行时间

##### 第1式：测试代码运行时间

In [1]:
# 平凡方法
import time

start = time.time()
job = [i**2 for i in range(1, 100000, 3)]
end = time.time()
print(f"Total used:{end - start}")

Total used:0.012769699096679688


In [3]:
# 魔法方法
%time
job = [i**2 for i in range(1, 100000, 3)]

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 12.2 µs


##### 第2式 测试代码多次运行平均时间

In [4]:
# 平凡方法
from timeit import timeit

g = lambda x:x**2+1

def main():
    return(g(2)**120)
timeit('main()', globals ={'main': main}, number = 10)

1.4513999985865667e-05

In [7]:
# 魔法方法
%timeit -n 10
g = lambda x:x**2+1
def main():
    return(g(2)**120)
main()

752316384526264005099991383822237233803945956334136013765601092018187046051025390625

##### 第3式：按调用函数分析代码运行时间

In [9]:
# 平凡方法
def relu(x):
    return x if x>0 else 0

def main():
    result = [relu(x) for x in range(-10000, 10000,1)]
    return result

In [10]:
import profile
profile.run('main()')

         20006 function calls in 0.134 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.133    0.133 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
    20000    0.062    0.000    0.062    0.000 <ipython-input-9-fb0cd65efd9d>:2(relu)
        1    0.000    0.000    0.133    0.133 <ipython-input-9-fb0cd65efd9d>:5(main)
        1    0.071    0.071    0.133    0.133 <ipython-input-9-fb0cd65efd9d>:6(<listcomp>)
        1    0.000    0.000    0.133    0.133 <string>:1(<module>)
        1    0.000    0.000    0.134    0.134 profile:0(main())
        0    0.000             0.000          profile:0(profiler)




In [11]:
# magic function
%prun main()

##### 第4式：按行分析代码运行时间

In [14]:
# 平凡方法
!pip install line_profiler

In [15]:
def relu(x):
    return x if x>0 else 0

def main():
    result = [relu(x) for x in range(-10000, 100000,1)]
    return result

In [ ]:
from line_profiler import LineProfiler
lprofile = LineProfiler(main, relu)
lprofile.run('main()')
lprofile.print_stats()

#### 二 、加速你的查找


##### 第5式: 用set而非list进行查找

In [35]:
# 低速方法
data = (i**2 +1 for i in range(1000000))
list_data = list(data)
set_data = set(data)

In [36]:
%time 
19999999 in list_data

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


False

In [37]:
# 高速方法
%time 
19999999 in set_data

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


False

##### 第6式：用dict而非两个list进行匹配查找

In [38]:
# 低速方法
list_a = [2*i-1 for i in range(100000)]
list_b = [i**2 for i in list_a]
dict_ab = dict(zip(list_a, list_b))

In [46]:
# 低速方法
%time
list_b[list_a.index(10101)]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


102030201

In [47]:
# 高速方法
%time 
dict_ab.get(10101,None)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


102030201

#### 三、加速你的循环

##### 第7式：优先使用for循环而不是while循环

In [48]:
# 低速方法
%time 
s, i = 0, 0
while i<10000:
    i += 1
    s += i
print(s)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs
50005000


In [50]:
# 高速方法
%time
s = 0
for i in range(1, 10001):
    s += i
print(s)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
50005000


##### 第8式：在循环体中避免重复计算

In [62]:
# 低速
a = [i**2 +1 for i in range(20000)]

%time
b = [i/sum(a) for i in a]

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


In [63]:
# 高速
sum_a = sum(a)

%time
b = [i/sum_a for i in a]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


#### 四、加速你的函数

##### 第9式：用循环机制代替递归函数 

In [74]:
# 低速
%time
def fib(n):
    return (1 if n in (1, 2) else fib(n-1)+fib(n-2))
fib(30)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


832040

In [75]:
# 高速
%time
def fib(n):
    if n in (1,2):
        return 1
    a, b = 1, 1
    for i in range(2, n):
        a, b = b, a+b
    return b
fib(30)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


832040

##### 第10式：用缓存机制加速递归函数

In [81]:
# 低速
%time
def fib(n):
    return 1 if n in (1,2) else fib(n-1)+fib(n-2)
fib(40)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11 µs


102334155

In [ ]:
# 高速
%time
from functools import lru_cache

@lru_cache(100)
def fib(n):
    return 1 if 1 in (1, 2) else fib(n-1)+fib(n-2)
fib(40)

##### 第11式：用numba加速Python函数

In [82]:
# 低速
%time
def my_power(x):
    return x**2

def my_power_sum(n):
    s = 0
    for i in range(1, n+1):
        s += my_power(i)
    return s

my_power_sum(100000)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs


333338333350000

In [83]:
# 高速
%time
from numba import jit

@jit
def my_power(x):
    return x**2

@jit
def my_power_sum(n):
    s = 0
    for i in range(1, n+1):
        s += my_power(i)
    return s

my_power_sum(100000)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


333338333350000

#### 五、使用标准库进行加速

##### 第12式：使用collections.Counter加速计数

In [88]:
# 低速
data = [x**2%1989 for x in range(2000000)]

%time
values_count = {}
for i in data:
    i_cnt = values_count.get(1,0)
    values_count[i] = i_cnt + 1
values_count.get(100, 0)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


8044

In [89]:
# 高速
%time
from collections import Counter
values_count = Counter(data)
values_count.get(100, 0)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


8044

##### 第13式 使用collections.ChainMap加速字典合并

In [90]:
# 低速
dict_a = {i:i+1 for i in range(1, 100000, 2)}
dict_b = {i:i+1 for i in range(1, 100000, 2)}
dict_c = {i:i+1 for i in range(1, 100000, 2)}
dict_d = {i:i+1 for i in range(1, 100000, 2)}

%time
result = dict_a.copy()
result.update(dict_b)
result.update(dict_c)
result.update(dict_d)
result.get(9999,0)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs


10000

In [91]:
# 高速
%time
from collections import ChainMap

chain = ChainMap(dict_a,dict_b, dict_c, dict_d)
chain.get(9999,0)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


10000

#### 六、使用高阶函数进行加速

##### 第14式：使用map代替推导式进行加速

In [99]:
# 低速
%time

result = [i**2 for i in range(1, 10000,2)]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [100]:
# 高速
%time
result = map(lambda x:x**2, range(1, 10000, 2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


##### 第15式：使用filter代替推导式进行加速

In [104]:
# 低速
%time

result = [x for x in range(1, 1000000, 3) if x%7 == 0]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs


In [103]:
# 高速
%time

result = filter(lambda x:x%7==0, range(1,1000000,3))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


#### 七、应用多进程多线程加速

##### 第16式：应用多线程加速IO密集型任务

In [ ]:
# 低速
%time
def write_file(i):
    with open(str(i)+ '.txt', 'w') as f:
        s = f"hello {i}" * 100000
        f.write(s)
        
# 串行任务
for i in range(10):
    write_file(i)

In [ ]:
# 高速
%time
import threading

def write_file(i):
    with open(str(i)+ '.txt', 'w') as f:
        s = f"hello {i}" * 100000
        f.write(s)

# 多线程任务
thread_list = []
for i in range(10):
    t = threading.Thread(target = write_file, args = (i,))
    t.setDaemon(True) # 设置为守护线程
    thread_list.append(t)
    
for t in thread_list:
    t.start() # 启动线程
    
for t in thread_list:
    t.join() # 等待子线程结束

##### 第17式：应用多进程加速CPU密集型任务

In [105]:
# 低速
%time

def job(x):
    time.sleep(5)
    return x**2

# 串行任务
ans = [job(i) for i in range(10)]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [106]:
# 高速
%time

import time
import multiprocessing

data = range(10)

def job(x):
    time.sleep(5)
    return x**2

# 多进程任务
pool = multiprocessing.Pool(processes = 4)
result = []
for i in range(10):
    result.append(pool.apply_async(job, (1,)))
pool.close()
pool.join()
ans = [res.get() for res in result]
ans

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]